参考: https://www.kaggle.com/yxohrxn/mlpclassifier-fit?scriptVersionId=46905918

In [1]:
import sys

#sys.path.append("../input/adabeliefoptimizer/pypi_packages/adabelief_tf0.1.0")

In [2]:
import sys

#sys.path.append("../input/iterative-stratification/iterative-stratification-master")

In [3]:
import os
import pickle

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
from tqdm import tqdm

In [4]:
import tensorflow as tf


def build_callbacks(
    model_path, factor=0.1, mode="auto", monitor="val_loss", patience=0, verbose=0
):
    early_stopping = tf.keras.callbacks.EarlyStopping(
        mode=mode, monitor=monitor, patience=patience, verbose=verbose
    )
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        model_path, mode=mode, monitor=monitor, save_best_only=True, verbose=verbose
    )
    reduce_lr_on_plateau = tf.keras.callbacks.ReduceLROnPlateau(
        factor=factor, monitor=monitor, mode=mode, verbose=verbose
    )

    return [early_stopping, model_checkpoint, reduce_lr_on_plateau]

In [5]:
import tensorflow as tf
import tensorflow_addons as tfa
from adabelief_tf import AdaBeliefOptimizer


def create_model_rs(shape1, shape2, n_class=206):
    """入力2つのNN.resnetみたくskip connection入れてる"""
    input_1 = tf.keras.layers.Input(shape=(shape1))
    input_2 = tf.keras.layers.Input(shape=(shape2))

    head_1 = tf.keras.layers.BatchNormalization()(input_1)
    head_1 = tf.keras.layers.Dropout(0.2)(head_1)
    head_1 = tf.keras.layers.Dense(512, activation="elu")(head_1)
    head_1 = tf.keras.layers.BatchNormalization()(head_1)
    input_3 = tf.keras.layers.Dense(256, activation="elu")(head_1)

    input_3_concat = tf.keras.layers.Concatenate()(
        [input_2, input_3]
    )  # node連結。node数が2つのnodeの足し算になる

    head_2 = tf.keras.layers.BatchNormalization()(input_3_concat)
    head_2 = tf.keras.layers.Dropout(0.3)(head_2)
    head_2 = tf.keras.layers.Dense(512, "relu")(head_2)
    head_2 = tf.keras.layers.BatchNormalization()(head_2)
    head_2 = tf.keras.layers.Dense(512, "elu")(head_2)
    head_2 = tf.keras.layers.BatchNormalization()(head_2)
    head_2 = tf.keras.layers.Dense(256, "relu")(head_2)
    head_2 = tf.keras.layers.BatchNormalization()(head_2)
    input_4 = tf.keras.layers.Dense(256, "elu")(head_2)

    input_4_avg = tf.keras.layers.Average()([input_3, input_4])  # 入力のリストを要素ごとに平均化

    head_3 = tf.keras.layers.BatchNormalization()(input_4_avg)
    head_3 = tf.keras.layers.Dense(
        256, kernel_initializer="lecun_normal", activation="selu"
    )(head_3)
    head_3 = tf.keras.layers.BatchNormalization()(head_3)
    head_3 = tf.keras.layers.Dense(
        n_class, kernel_initializer="lecun_normal", activation="selu"
    )(head_3)
    head_3 = tf.keras.layers.BatchNormalization()(head_3)
    output = tf.keras.layers.Dense(n_class, activation="sigmoid")(head_3)

    model = tf.keras.models.Model(inputs=[input_1, input_2], outputs=output)
    #rs_lr = 0.03  # C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\notebook\base_line\20201105
    #print(f"rs_lr: {rs_lr}")
    opt = tf.optimizers.Adam(learning_rate=params["lr"])
    model.compile(
        optimizer=opt,
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.0015),  # ラベルスムージング
        metrics=tf.keras.metrics.BinaryCrossentropy(),
    )
    # model.save("model/rs_shape.h5")
    return model


def create_model_5l(shape, n_class=206):
    """入力1つの5層NN。Stochastic Weight Averaging使う"""
    inp = tf.keras.layers.Input(shape=(shape))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(2560, activation="relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(2048, activation="relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(1524, activation="relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(1012, activation="relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(780, activation="relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    out = tf.keras.layers.Dense(n_class, activation="sigmoid")(x)
    model = tf.keras.models.Model(inputs=inp, outputs=out)
    opt = tf.optimizers.Adam(learning_rate=params["lr"])
    opt = tfa.optimizers.SWA(
        opt
    )  # Stochastic Weight Averaging.モデルの重みを、これまで＋今回の平均を取って更新していくことでうまくいくみたい https://twitter.com/icoxfog417/status/989762534163992577
    model.compile(
        optimizer=opt,
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.0020),  # ラベルスムージング
        metrics=tf.keras.metrics.BinaryCrossentropy(),
    )
    # model.save("model/5l_shape.h5")
    return model


def create_model_4l(shape, n_class=206):
    """入力1つの4層NN。シンプル"""
    inp = tf.keras.layers.Input(shape=(shape))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(2048, activation="relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(1524, activation="relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(1012, activation="relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(1012, activation="relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    out = tf.keras.layers.Dense(n_class, activation="sigmoid")(x)
    model = tf.keras.models.Model(inputs=inp, outputs=out)
    opt = tf.optimizers.Adam(learning_rate=params["lr"])
    model.compile(
        optimizer=opt,
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.0020),  # ラベルスムージング
        metrics=tf.keras.metrics.BinaryCrossentropy(),
    )
    # model.save("model/4l_shape.h5")
    return model


def create_model_3l_v2(shape, n_class=206):
    """入力1つの3層NN。WeightNormalization adabelief_tf 使う"""
    inp = tf.keras.layers.Input(shape=(shape))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(0.4914099166744246)(x)
    x = tfa.layers.WeightNormalization(tf.keras.layers.Dense(1159, activation="relu"))(
        x
    )
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.18817607797795838)(x)
    x = tfa.layers.WeightNormalization(tf.keras.layers.Dense(960, activation="relu"))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.12542057776853896)(x)
    x = tfa.layers.WeightNormalization(tf.keras.layers.Dense(1811, activation="relu"))(
        x
    )
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.20175242230280122)(x)
    out = tfa.layers.WeightNormalization(
        tf.keras.layers.Dense(n_class, activation="sigmoid")
    )(x)
    model = tf.keras.models.Model(inputs=inp, outputs=out)
    #_lr = 0.01  # C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\notebook\base_line\20201105
    #print(f"3l_v2_lr: {_lr}")
    opt = AdaBeliefOptimizer(learning_rate=params["lr"])
    model.compile(
        optimizer=opt,
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.0015),
        metrics=tf.keras.metrics.BinaryCrossentropy(),
    )
    # model.save("model/3l_shape.h5")
    return model


def create_model_2l(shape, n_class=206):
    """入力1つの2層NN。Lookahead, WeightNormalization使う"""
    inp = tf.keras.layers.Input(shape=(shape))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(0.2688628097505064)(x)
    x = tfa.layers.WeightNormalization(tf.keras.layers.Dense(1292, activation="relu"))(
        x
    )
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.4598218403250696)(x)
    x = tfa.layers.WeightNormalization(tf.keras.layers.Dense(983, activation="relu"))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.4703144018483698)(x)
    out = tfa.layers.WeightNormalization(
        tf.keras.layers.Dense(n_class, activation="sigmoid")
    )(x)
    model = tf.keras.models.Model(inputs=inp, outputs=out)
    #_lr = 0.1  # C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\notebook\base_line\20201105
    #print(f"2l_lr: {_lr}")
    opt = tf.optimizers.Adam(learning_rate=params["lr"])
    opt = tfa.optimizers.Lookahead(opt, sync_period=10)
    model.compile(
        optimizer=opt,
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.0015),
        metrics=tf.keras.metrics.BinaryCrossentropy(),
    )
    # model.save("model/2l_shape.h5")
    return model

In [6]:
# def compute_absolute_features(X):
#     X = X.abs()

#     return X.rename(columns="{}_abs".format)

In [7]:
# def compute_square_features(X):
#     X = X ** 2

#     return X.rename(columns="{}_square".format)

In [8]:
# import pandas as pd


# def compute_row_statistics(X, prefix=""):
#     Xt = pd.DataFrame()

#     for agg_func in [
#         # "min",
#         # "max",
#         "mean",
#         "std",
#         "kurtosis",
#         "skew",
#     ]:
#         Xt[f"{prefix}{agg_func}"] = X.agg(agg_func, axis=1)

#     return Xt

In [9]:
from sklearn.metrics import log_loss


def score(Y, Y_pred):
    _, n_classes = Y.shape

    losses = []

    for j in range(n_classes):
        loss = log_loss(Y.iloc[:, j], Y_pred.iloc[:, j], labels=[0, 1])

        losses.append(loss)

    return np.mean(losses)

In [10]:
import os
import random as rn

import tensorflow as tf
import numpy as np


def set_seed(seed=0):
    os.environ["PYTHONHASHSEED"] = str(seed)

    rn.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    graph = tf.compat.v1.get_default_graph()
    session_conf = tf.compat.v1.ConfigProto(
        inter_op_parallelism_threads=1, intra_op_parallelism_threads=1
    )
    sess = tf.compat.v1.Session(graph=graph, config=session_conf)

    tf.compat.v1.keras.backend.set_session(sess)

In [11]:
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin


class ClippedFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, copy=True, high=0.99, low=0.01):
        self.copy = copy
        self.high = high
        self.low = low

    def fit(self, X, y=None):
        self.data_max_ = X.quantile(q=self.high)
        self.data_min_ = X.quantile(q=self.low)

        return self

    def transform(self, X):
        if self.copy:
            X = X.copy()

        X.clip(self.data_min_, self.data_max_, axis=1, inplace=True)

        return X

In [12]:
# https://arxiv.org/abs/1905.04899

import numpy as np
import tensorflow as tf


class Cutmix(tf.keras.utils.Sequence):
    def __init__(self, X, y=None, batch_size=32, alpha=1.0):
        self.X = np.asarray(X)

        if y is None:
            self.y = y
        else:
            self.y = np.asarray(y)

        self.batch_size = batch_size
        self.alpha = alpha

    def __getitem__(self, i):
        X_batch = self.X[i * self.batch_size : (i + 1) * self.batch_size]

        n_samples, n_features = self.X.shape
        batch_size = X_batch.shape[0]
        shuffle = np.random.choice(n_samples, batch_size)

        l = np.random.beta(self.alpha, self.alpha)
        mask = np.random.choice([0.0, 1.0], size=n_features, p=[1.0 - l, l])
        X_shuffle = self.X[shuffle]
        X_batch = mask * X_batch + (1.0 - mask) * X_shuffle

        if self.y is None:
            return X_batch, None

        y_batch = self.y[i * self.batch_size : (i + 1) * self.batch_size]
        y_shuffle = self.y[shuffle]
        y_batch = l * y_batch + (1.0 - l) * y_shuffle

        return X_batch, y_batch

    def __len__(self):
        n_samples = self.X.shape[0]

        return int(np.ceil(n_samples / self.batch_size))

In [13]:
import numpy as np
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection._split import _BaseKFold


class MultilabelStratifiedGroupKFold(_BaseKFold):
    def __init__(self, n_splits=5, random_state=None, shuffle=False):
        super().__init__(n_splits=n_splits, random_state=random_state, shuffle=shuffle)

    def _iter_test_indices(self, X=None, y=None, groups=None):
        cv = MultilabelStratifiedKFold(
            n_splits=self.n_splits,
            random_state=self.random_state,
            shuffle=self.shuffle,
        )

        value_counts = groups.value_counts()
        regluar_indices = value_counts.loc[
            (value_counts == 6) | (value_counts == 12) | (value_counts == 18)
        ].index.sort_values()
        irregluar_indices = value_counts.loc[
            (value_counts != 6) & (value_counts != 12) & (value_counts != 18)
        ].index.sort_values()

        group_to_fold = {}
        tmp = Y.groupby(groups).mean().loc[regluar_indices]

        for fold, (_, test) in enumerate(cv.split(tmp, tmp)):
            group_to_fold.update({group: fold for group in tmp.index[test]})

        sample_to_fold = {}
        tmp = Y.loc[groups.isin(irregluar_indices)]

        for fold, (_, test) in enumerate(cv.split(tmp, tmp)):
            sample_to_fold.update({sample: fold for sample in tmp.index[test]})

        folds = groups.map(group_to_fold)
        is_na = folds.isna()
        folds[is_na] = folds[is_na].index.map(sample_to_fold).values

        for i in range(self.n_splits):
            yield np.where(folds == i)[0]

In [14]:
#dtype = {"cp_type": "category", "cp_dose": "category"}
#index_col = "sig_id"
#
#train_features = pd.read_csv(
#    "../input/lish-moa/train_features.csv", dtype=dtype, index_col=index_col
#)
#X = train_features.select_dtypes("number")
## Y_nonscored = pd.read_csv(
##     "../input/lish-moa/train_targets_nonscored.csv", index_col=index_col
## )
#Y = pd.read_csv("../input/lish-moa/train_targets_scored.csv", index_col=index_col)
#groups = pd.read_csv(
#    "../input/lish-moa/train_drug.csv", index_col=index_col, squeeze=True
#)
#
#columns = Y.columns

In [15]:
dtype = {"cp_type": "category", "cp_dose": "category"}
index_col = "sig_id"

sys.path.append(
    r"C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\code"
)
import datasets

DATADIR = datasets.DATADIR

groups = pd.read_csv(f"{DATADIR}/train_drug.csv", dtype=dtype, index_col=index_col, squeeze=True)
train_features = pd.read_csv(f"{DATADIR}/train_features.csv", dtype=dtype, index_col=index_col)
#X_test = pd.read_csv(f"{DATADIR}/test_features.csv", dtype=dtype, index_col=index_col)
X = train_features.select_dtypes("number")
Y_nonscored = pd.read_csv(f"{DATADIR}/train_targets_nonscored.csv", index_col=index_col)
Y = pd.read_csv(f"{DATADIR}/train_targets_scored.csv", index_col=index_col)

columns = Y.columns

In [16]:
clipped_features = ClippedFeatures()
X = clipped_features.fit_transform(X)

with open("clipped_features.pkl", "wb") as f:
    pickle.dump(clipped_features, f)

# c_prefix = "c-"
# g_prefix = "g-"
# c_columns = X.columns.str.startswith(c_prefix)
# g_columns = X.columns.str.startswith(g_prefix)
# X_c = compute_row_statistics(X.loc[:, c_columns], prefix=c_prefix)
# X_g = compute_row_statistics(X.loc[:, g_columns], prefix=g_prefix)
# X = pd.concat([X, X_c, X_g], axis=1)

# Y_nonscored = Y_nonscored.loc[:, Y_nonscored.sum(axis=0) > 0]

# Y = pd.concat([Y, Y_nonscored], axis=1)

In [17]:
train_size, n_features = X.shape
# _, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

In [18]:
def train_and_evaluate(model_type="rs"):
    counts = np.empty((n_seeds * n_splits, n_classes))

    bias_initializer = -Y.mean(axis=0).apply(np.log).values
    bias_initializer = tf.keras.initializers.Constant(bias_initializer)

    Y_pred = np.zeros((train_size, n_classes))
    Y_pred = pd.DataFrame(Y_pred, columns=Y.columns, index=Y.index)

    for i in range(n_seeds):
        set_seed(seed=i)

        cv = MultilabelStratifiedGroupKFold(
            n_splits=n_splits, random_state=i, shuffle=shuffle
        )

        for j, (train, valid) in enumerate(cv.split(X, Y, groups)):
            counts[i * n_splits + j] = Y.iloc[train].sum()

            os.makedirs(model_type, exist_ok=True)
            
            # model_nonscored_path = f"model_nonscored_seed_{i}_fold_{j}.h5"
            model_path = f"{model_type}/model_seed_{i}_fold_{j}.h5"

            K.clear_session()
            if model_type == "5l":
                model = create_model_5l(X.shape[1], n_class=len(columns))
            elif model_type == "4l":
                model = create_model_4l(X.shape[1], n_class=len(columns))
            elif model_type == "3l_v2":
                model = create_model_3l_v2(X.shape[1], n_class=len(columns))
            elif model_type == "2l":
                model = create_model_2l(X.shape[1], n_class=len(columns))
            elif model_type == "rs":
                model = create_model_rs(X.shape[1], len(start_predictors), n_class=len(columns))
            
            if model_type == "rs":
                # 入力2つのNN使うから工夫してる
                X_ = X[start_predictors]

                callbacks = build_callbacks(model_path, factor=factor, patience=patience)
                history = model.fit(
                    [X.iloc[train], X_.iloc[train]],
                    Y.iloc[train],
                    batch_size=batch_size,
                    callbacks=callbacks,
                    validation_data=([X.iloc[valid], X_.iloc[valid]], Y.iloc[valid]),
                    **fit_params,
                )
                
                model.load_weights(model_path)

                Y_pred.iloc[valid] += model.predict([X.iloc[valid], X_.iloc[valid]]) / n_seeds
            
            else:
                generator = Cutmix(
                    X.iloc[train], Y.iloc[train], alpha=alpha, batch_size=batch_size
                )
                callbacks = build_callbacks(model_path, factor=factor, patience=patience)
                history = model.fit(
                    generator,
                    callbacks=callbacks,
                    validation_data=(X.iloc[valid], Y.iloc[valid]),
                    **fit_params,
                )

                model.load_weights(model_path)

                Y_pred.iloc[valid] += model.predict(X.iloc[valid]) / n_seeds

    Y_pred[train_features["cp_type"] == "ctl_vehicle"] = 0.0

    with open("counts.pkl", "wb") as f:
        pickle.dump(counts, f)

    with open(f"Y_pred_{model_type}.pkl", "wb") as f:
        pickle.dump(Y_pred[columns], f)
        
    oof_score = score(Y[columns], Y_pred[columns])
    print(f"oof_score: {oof_score}")
    print("-" * 100)
    
    return oof_score, Y_pred

In [19]:
# hyperparameters
alpha = 4.0
batch_size = 32
factor = 0.5
n_seeds = 5
n_splits = 5
patience = 30
shuffle = True
params = {
    "activation": "elu",
    "kernel_initializer": "he_normal",
    "label_smoothing": 5e-04,
    "lr": 0.03,
    "n_layers": 6,
    "n_units": 256,
    "rate": 0.3,
}
fit_params = {"epochs": 1_000, "verbose": 0}


#DEBUG = True
DEBUG = False
if DEBUG:
    batch_size = 1024
    n_seeds = 2
    n_splits = 2
    fit_params = {"epochs": 2, "verbose": 1}
    print("DEBUG")

In [20]:
start_predictors = [
    "g-0",
    "g-7",
    "g-8",
    "g-10",
    "g-13",
    "g-17",
    "g-20",
    "g-22",
    "g-24",
    "g-26",
    "g-28",
    "g-29",
    "g-30",
    "g-31",
    "g-32",
    "g-34",
    "g-35",
    "g-36",
    "g-37",
    "g-38",
    "g-39",
    "g-41",
    "g-46",
    "g-48",
    "g-50",
    "g-51",
    "g-52",
    "g-55",
    "g-58",
    "g-59",
    "g-61",
    "g-62",
    "g-63",
    "g-65",
    "g-66",
    "g-67",
    "g-68",
    "g-70",
    "g-72",
    "g-74",
    "g-75",
    "g-79",
    "g-83",
    "g-84",
    "g-85",
    "g-86",
    "g-90",
    "g-91",
    "g-94",
    "g-95",
    "g-96",
    "g-97",
    "g-98",
    "g-100",
    "g-102",
    "g-105",
    "g-106",
    "g-112",
    "g-113",
    "g-114",
    "g-116",
    "g-121",
    "g-123",
    "g-126",
    "g-128",
    "g-131",
    "g-132",
    "g-134",
    "g-135",
    "g-138",
    "g-139",
    "g-140",
    "g-142",
    "g-144",
    "g-145",
    "g-146",
    "g-147",
    "g-148",
    "g-152",
    "g-155",
    "g-157",
    "g-158",
    "g-160",
    "g-163",
    "g-164",
    "g-165",
    "g-170",
    "g-173",
    "g-174",
    "g-175",
    "g-177",
    "g-178",
    "g-181",
    "g-183",
    "g-185",
    "g-186",
    "g-189",
    "g-192",
    "g-194",
    "g-195",
    "g-196",
    "g-197",
    "g-199",
    "g-201",
    "g-202",
    "g-206",
    "g-208",
    "g-210",
    "g-213",
    "g-214",
    "g-215",
    "g-220",
    "g-226",
    "g-228",
    "g-229",
    "g-235",
    "g-238",
    "g-241",
    "g-242",
    "g-243",
    "g-244",
    "g-245",
    "g-248",
    "g-250",
    "g-251",
    "g-254",
    "g-257",
    "g-259",
    "g-261",
    "g-266",
    "g-270",
    "g-271",
    "g-272",
    "g-275",
    "g-278",
    "g-282",
    "g-287",
    "g-288",
    "g-289",
    "g-291",
    "g-293",
    "g-294",
    "g-297",
    "g-298",
    "g-301",
    "g-303",
    "g-304",
    "g-306",
    "g-308",
    "g-309",
    "g-310",
    "g-311",
    "g-314",
    "g-315",
    "g-316",
    "g-317",
    "g-320",
    "g-321",
    "g-322",
    "g-327",
    "g-328",
    "g-329",
    "g-332",
    "g-334",
    "g-335",
    "g-336",
    "g-337",
    "g-339",
    "g-342",
    "g-344",
    "g-349",
    "g-350",
    "g-351",
    "g-353",
    "g-354",
    "g-355",
    "g-357",
    "g-359",
    "g-360",
    "g-364",
    "g-365",
    "g-366",
    "g-367",
    "g-368",
    "g-369",
    "g-374",
    "g-375",
    "g-377",
    "g-379",
    "g-385",
    "g-386",
    "g-390",
    "g-392",
    "g-393",
    "g-400",
    "g-402",
    "g-406",
    "g-407",
    "g-409",
    "g-410",
    "g-411",
    "g-414",
    "g-417",
    "g-418",
    "g-421",
    "g-423",
    "g-424",
    "g-427",
    "g-429",
    "g-431",
    "g-432",
    "g-433",
    "g-434",
    "g-437",
    "g-439",
    "g-440",
    "g-443",
    "g-449",
    "g-458",
    "g-459",
    "g-460",
    "g-461",
    "g-464",
    "g-467",
    "g-468",
    "g-470",
    "g-473",
    "g-477",
    "g-478",
    "g-479",
    "g-484",
    "g-485",
    "g-486",
    "g-488",
    "g-489",
    "g-491",
    "g-494",
    "g-496",
    "g-498",
    "g-500",
    "g-503",
    "g-504",
    "g-506",
    "g-508",
    "g-509",
    "g-512",
    "g-522",
    "g-529",
    "g-531",
    "g-534",
    "g-539",
    "g-541",
    "g-546",
    "g-551",
    "g-553",
    "g-554",
    "g-559",
    "g-561",
    "g-562",
    "g-565",
    "g-568",
    "g-569",
    "g-574",
    "g-577",
    "g-578",
    "g-586",
    "g-588",
    "g-590",
    "g-594",
    "g-595",
    "g-596",
    "g-597",
    "g-599",
    "g-600",
    "g-603",
    "g-607",
    "g-615",
    "g-618",
    "g-619",
    "g-620",
    "g-625",
    "g-628",
    "g-629",
    "g-632",
    "g-634",
    "g-635",
    "g-636",
    "g-638",
    "g-639",
    "g-641",
    "g-643",
    "g-644",
    "g-645",
    "g-646",
    "g-647",
    "g-648",
    "g-663",
    "g-664",
    "g-665",
    "g-668",
    "g-669",
    "g-670",
    "g-671",
    "g-672",
    "g-673",
    "g-674",
    "g-677",
    "g-678",
    "g-680",
    "g-683",
    "g-689",
    "g-691",
    "g-693",
    "g-695",
    "g-701",
    "g-702",
    "g-703",
    "g-704",
    "g-705",
    "g-706",
    "g-708",
    "g-711",
    "g-712",
    "g-720",
    "g-721",
    "g-723",
    "g-724",
    "g-726",
    "g-728",
    "g-731",
    "g-733",
    "g-738",
    "g-739",
    "g-742",
    "g-743",
    "g-744",
    "g-745",
    "g-749",
    "g-750",
    "g-752",
    "g-760",
    "g-761",
    "g-764",
    "g-766",
    "g-768",
    "g-770",
    "g-771",
    "c-0",
    "c-1",
    "c-2",
    "c-3",
    "c-4",
    "c-5",
    "c-6",
    "c-7",
    "c-8",
    "c-9",
    "c-10",
    "c-11",
    "c-12",
    "c-13",
    "c-14",
    "c-15",
    "c-16",
    "c-17",
    "c-18",
    "c-19",
    "c-20",
    "c-21",
    "c-22",
    "c-23",
    "c-24",
    "c-25",
    "c-26",
    "c-27",
    "c-28",
    "c-29",
    "c-30",
    "c-31",
    "c-32",
    "c-33",
    "c-34",
    "c-35",
    "c-36",
    "c-37",
    "c-38",
    "c-39",
    "c-40",
    "c-41",
    "c-42",
    "c-43",
    "c-44",
    "c-45",
    "c-46",
    "c-47",
    "c-48",
    "c-49",
    "c-50",
    "c-51",
    "c-52",
    "c-53",
    "c-54",
    "c-55",
    "c-56",
    "c-57",
    "c-58",
    "c-59",
    "c-60",
    "c-61",
    "c-62",
    "c-63",
    "c-64",
    "c-65",
    "c-66",
    "c-67",
    "c-68",
    "c-69",
    "c-70",
    "c-71",
    "c-72",
    "c-73",
    "c-74",
    "c-75",
    "c-76",
    "c-77",
    "c-78",
    "c-79",
    "c-80",
    "c-81",
    "c-82",
    "c-83",
    "c-84",
    "c-85",
    "c-86",
    "c-87",
    "c-88",
    "c-89",
    "c-90",
    "c-91",
    "c-92",
    "c-93",
    "c-94",
    "c-95",
    "c-96",
    "c-97",
    "c-98",
    "c-99",
]

In [21]:
%%time

for model_type in tqdm(["5l", "4l", "3l_v2", "2l", "rs"]):
    oof_score, Y_pred = train_and_evaluate(model_type=model_type)

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=2 as keyword args. From version 0.25 

oof_score: 0.01768391246506444
----------------------------------------------------------------------------------------------------


c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=2 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.

oof_score: 0.01765186754937085
----------------------------------------------------------------------------------------------------


c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


...

oof_score: 0.01597963119837256
----------------------------------------------------------------------------------------------------


c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=2 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.

oof_score: 0.01580595608041927
----------------------------------------------------------------------------------------------------


c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
c:\users\81908\appdata\local\pypoetry\cache\virtualenvs\tfgpu-ehdmne1y-py3.8\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass shuffle=True, random_state=2 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.

oof_score: 0.01615753132792794
----------------------------------------------------------------------------------------------------
Wall time: 12h 56min 45s


# pkl check

In [22]:
path = r"Y_pred_rs.pkl"
with open(path, 'rb') as f:
    Y_pred = pickle.load(f)
Y_pred

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
sig_id,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,0.002047,0.000838,0.001295,0.007527,0.013149,0.004847,0.002490,0.006570,0.000903,0.011098,...,0.000949,0.000881,0.002373,0.001060,0.002394,0.000712,0.002275,0.001261,0.001185,0.001974
id_000779bfc,0.002455,0.000928,0.001763,0.011832,0.013628,0.004554,0.003857,0.003473,0.001785,0.038254,...,0.001590,0.001552,0.004078,0.001070,0.001621,0.000729,0.001231,0.002465,0.005594,0.002211
id_000a6266a,0.002566,0.003215,0.002172,0.004016,0.007998,0.004084,0.002654,0.010015,0.001308,0.014530,...,0.001009,0.001883,0.001539,0.007104,0.015232,0.001074,0.112120,0.001844,0.000942,0.000988
id_0015fd391,0.000530,0.000994,0.001246,0.006408,0.003324,0.004299,0.000743,0.003454,0.000604,0.001440,...,0.000895,0.020174,0.001939,0.262665,0.008578,0.001071,0.000789,0.002153,0.000955,0.000966
id_001626bd3,0.001169,0.002075,0.002269,0.020104,0.017392,0.006510,0.002628,0.003249,0.000815,0.012420,...,0.001089,0.003586,0.003740,0.001849,0.001505,0.001130,0.002606,0.003532,0.001917,0.001427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_fffb1ceed,0.002346,0.002166,0.001557,0.008122,0.020252,0.004821,0.002126,0.007345,0.000670,0.017258,...,0.000803,0.001586,0.002495,0.003380,0.002166,0.001103,0.004194,0.001240,0.002227,0.001214
id_fffb70c0c,0.002841,0.002073,0.001806,0.010787,0.019011,0.002581,0.003442,0.006740,0.001310,0.016394,...,0.000773,0.001906,0.003343,0.002922,0.006125,0.001084,0.003718,0.001687,0.002884,0.002661
id_fffc1c3f4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# predict test

In [23]:
model = tf.keras.models.load_model("3l_v2/model_seed_0_fold_0.h5")
model.predict(X)

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



array([[0.00097113, 0.00099923, 0.00164678, ..., 0.00228117, 0.00047049,
        0.00388363],
       [0.00050459, 0.00094136, 0.00204052, ..., 0.0022702 , 0.00137131,
        0.00728543],
       [0.00176283, 0.00238633, 0.00148891, ..., 0.00090021, 0.00090038,
        0.00071903],
       ...,
       [0.00058132, 0.00105596, 0.00175163, ..., 0.00139925, 0.0018929 ,
        0.00431458],
       [0.00047458, 0.0005253 , 0.00152233, ..., 0.00114813, 0.00028543,
        0.00131423],
       [0.00101256, 0.00173873, 0.00125947, ..., 0.00106908, 0.00082397,
        0.00070895]], dtype=float32)

In [24]:
model = tf.keras.models.load_model("rs/model_seed_0_fold_0.h5")
model.predict([X, X[start_predictors]])

array([[5.9563238e-03, 6.3524037e-03, 4.9401144e-04, ..., 3.5287146e-04,
        1.6071307e-03, 2.0902290e-03],
       [2.8242467e-03, 1.3125869e-03, 2.3694434e-03, ..., 8.2034233e-04,
        9.8510049e-03, 1.3050347e-03],
       [3.2353771e-03, 4.9278950e-03, 6.0086098e-04, ..., 2.6966780e-04,
        1.0237792e-03, 1.0014391e-03],
       ...,
       [3.0501022e-03, 2.8825360e-03, 1.7978727e-03, ..., 7.3233899e-04,
        2.0559898e-03, 1.9992630e-03],
       [4.7921363e-04, 9.8579898e-05, 1.9377201e-04, ..., 4.0720861e-05,
        6.8192254e-03, 2.3727336e-05],
       [2.3289090e-03, 3.8564040e-03, 1.8716557e-03, ..., 6.8119442e-04,
        1.1835402e-03, 1.3261788e-03]], dtype=float32)